In [3]:
import datetime
import os
import sys
import pandas as pd
import FinanceDataReader as fdr

from tabulate import tabulate
from datetime import timedelta

df_kospi = fdr.StockListing('KOSPI')
df_kosdaq = fdr.StockListing('KOSDAQ')

# 관리종목
df_AdmStock = fdr.StockListing('KRX-ADMIN')

df_krx = pd.concat([df_kospi, df_kosdaq])

# 관리종목 및 우선주 제거(5, 7, 9, K)
df_filterStk = df_krx[~df_krx['Code'].isin(df_AdmStock['Symbol'])]
df_filterStk = df_filterStk[~df_filterStk['Code'].str.endswith(('5', '7', '9', 'K'))]

df_filterStk[0:10]

,Code,ISU_CD,Name,Market,Dept,Close,ChangeCode,Changes,ChagesRatio,Open,High,Low,Volume,Amount,Marcap,Stocks,MarketId
0,005930,KR7005930003,삼성전자,KOSPI,,78100,2,-1500,-1.88,79200,79500,78000,16123051,1267630623100,466240017155000,5969782550,STK
1,000660,KR7000660001,SK하이닉스,KOSPI,,223000,1,2000,0.90,219000,229500,219000,2198340,493039410000,162344527395000,728002365,STK
2,373220,KR7373220003,LG에너지솔루션,KOSPI,,336500,2,-12500,-3.58,345000,346000,336500,273382,92696426500,78741000000000,234000000,STK
3,005380,KR7005380001,현대차,KOSPI,,278500,1,10500,3.92,276000,285000,275500,1945849,546090330000,58322409193500,209416191,STK
4,207940,KR7207940008,삼성바이오로직스,KOSPI,,736000,2,-1000,-0.14,735000,737000,730000,37449,27426491000,52384064000000,71174000,STK
5,000270,KR7000270009,기아,KOSPI,,129100,1,6400,5.22,124000,129400,123800,2654199,337898780800,51621721634700,399858417,STK
7,068270,KR7068270008,셀트리온,KOSPI,,179800,2,-3600,-1.96,183400,183900,179700,604921,109175557000,39015381495400,216993223,STK
8,105560,KR7105560007,KB금융,KOSPI,,78300,2,-2600,-3.21,79600,80500,78000,955490,75203208700,31594916937600,403511072,STK
9,005490,KR7005490008,POSCO홀딩스,KOSPI,,371000,2,-9000,-2.37,376000,376500,370500,389824,145188041500,31375926330000,84571230,STK
10,035420,KR7035420009,NAVER,KOSPI,,166100,2,-3800,-2.24,168800,169400,166000,876593,146203435500,26976067463400,162408594,STK


In [4]:
df_filterStk['BF_HALF_YEAR_DATE'] = None
df_filterStk['BF_HALF_YEAR_CLOSE'] = None
df_filterStk['HALF_YEAR_RATIO'] = 0

initDate = datetime.date(2018, 12, 28)
bfDate = initDate + timedelta(days=-180)    # 6개월 전 날짜(2018-07-01)


for index, row in df_filterStk.iterrows():
    stkCode = row['Code']
    
    try:
        # df_stockPrice = fdr.DataReader(stkCode, bfDate, initDate)
        df_stockPrice = fdr.DataReader(stkCode, bfDate, initDate) if fdr.DataReader(stkCode, bfDate, initDate).shape[0] > 0 else fdr.DataReader(stkCode)
        bfPrice = df_stockPrice.iloc[0]['Close']
        currPrice = df_stockPrice.iloc[-1]['Close']
        
        # print(f'[{stkCode}] before Date : {df_stockPrice.iloc[0].name.strftime('%Y-%m-%d')}, {bfPrice} | curr Date : {df_stockPrice.iloc[-1].name.strftime('%Y-%m-%d')}, {currPrice}')
        df_filterStk.loc[df_filterStk['Code'] == stkCode, 'BF_HALF_YEAR_DATE'] = bfDate
        df_filterStk.loc[df_filterStk['Code'] == stkCode, 'BF_HALF_YEAR_CLOSE'] = bfPrice
        df_filterStk.loc[df_filterStk['Code'] == stkCode, 'HALF_YEAR_RATIO'] = (currPrice - bfPrice) / bfPrice * 100 
    except Exception as e:
        print(f'{e.args}')


filtered_df = df_filterStk[df_filterStk['HALF_YEAR_RATIO'] >= 10]
filtered_df

C:\Users\kimda\AppData\Local\Temp\ipykernel_37016\834086919.py:21: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '-15.0384193194292' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  df_filterStk.loc[df_filterStk['Code'] == stkCode, 'HALF_YEAR_RATIO'] = (currPrice - bfPrice) / bfPrice * 100


,Code,ISU_CD,Name,Market,Dept,Close,ChangeCode,Changes,ChagesRatio,Open,High,Low,Volume,Amount,Marcap,Stocks,MarketId,BF_HALF_YEAR_DATE,BF_HALF_YEAR_CLOSE,HALF_YEAR_RATIO
16,003670,KR7003670007,포스코퓨처엠,KOSPI,,263500,2,-9500,-3.48,268000,271500,263000,325863,86723640000,20411558470000,77463220,STK,2018-07-01,41345.0,44.609989
30,402340,KR7402340004,SK스퀘어,KOSPI,,90700,2,-700,-0.77,90100,91800,88000,317838,28770796100,12221821372000,134749960,STK,2018-07-01,76000.0,19.342105
31,012450,KR7012450003,한화에어로스페이스,KOSPI,,238500,1,10000,4.38,232000,241500,228000,687722,163298634000,12075255000000,50630000,STK,2018-07-01,22500.0,44.444444
33,329180,KR7329180004,HD현대중공업,KOSPI,,134500,1,1900,1.43,132600,135500,130900,142608,19125876000,11939984102000,88773116,STK,2018-07-01,111500.0,20.627803
37,017670,KR7017670001,SK텔레콤,KOSPI,,51000,1,100,0.20,50800,51100,50700,522604,26584486500,10954292703000,214790053,STK,2018-07-01,39770.0,16.919789
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1703,442770,KR7442770004,IBKS제21호스팩,KOSDAQ,SPAC(소속부없음),2410,1,25,1.05,2380,2410,2380,1926,4604455,10146100000,4210000,KSQ,2018-07-01,2000.0,20.500000
1708,440790,KR7440790004,교보13호스팩,KOSDAQ,SPAC(소속부없음),2365,1,20,0.85,2355,2365,2350,3066,7215725,9578250000,4050000,KSQ,2018-07-01,1975.0,19.746835
1709,464440,KR7464440007,한국제13호스팩,KOSDAQ,SPAC(소속부없음),2205,1,10,0.46,2175,2205,2175,25380,55629395,9525600000,4320000,KSQ,2018-07-01,1980.0,11.363636
1710,418210,KR7418210001,신한제10호스팩,KOSDAQ,SPAC(소속부없음),2600,1,100,4.00,2485,2605,2485,21778,55972210,9516000000,3660000,KSQ,2018-07-01,2145.0,21.212121


In [17]:
top_20_df = filtered_df.nlargest(20, 'HALF_YEAR_RATIO')
top_20_df


test = fdr.DataReader('247540')
test

,Code,ISU_CD,Name,Market,Dept,Close,ChangeCode,Changes,ChagesRatio,Open,High,Low,Volume,Amount,Marcap,Stocks,MarketId,BF_HALF_YEAR_DATE,BF_HALF_YEAR_CLOSE,HALF_YEAR_RATIO
0,247540,KR7247540008,에코프로비엠,KOSDAQ GLOBAL,우량기업부,198900,2,-1600,-0.80,195300,201000,193800,547470,108082783300,19452687321600,97801344,KSQ,2018-07-01,15616.0,1173.693648
10,257720,KR7257720003,실리콘투,KOSDAQ,벤처기업부,49050,2,-2150,-4.20,50900,51000,47600,1758506,86602396550,2962091927700,60389234,KSQ,2018-07-01,4833.0,914.897579
7,277810,KR7277810008,레인보우로보틱스,KOSDAQ,기술성장기업부,167700,1,3700,2.26,174000,174300,166300,289463,49300424900,3253356186600,19399858,KSQ,2018-07-01,26000.0,545.000000
108,114840,KR7114840002,아이패밀리에스씨,KOSDAQ,벤처기업부,39850,2,-1350,-3.28,42950,42950,39300,241365,9706414950,685497468400,17201944,KSQ,2018-07-01,6351.0,527.460242
193,336370,KR7336370002,솔루스첨단소재,KOSPI,,16410,2,-630,-3.70,17040,17140,16410,340890,5677010580,1152266615040,70217344,STK,2018-07-01,2655.0,518.079096
118,335890,KR7335890000,비올,KOSDAQ,벤처기업부,10550,1,190,1.83,10460,10730,10410,1458608,15409245420,616321768750,58419125,KSQ,2018-07-01,2191.0,381.515290
547,353810,KR7353810005,이지바이오,KOSDAQ,우량기업부,4615,1,5,0.11,4680,4685,4500,118478,545077135,157794787800,34191720,KSQ,2018-07-01,1017.0,353.785644
169,336260,KR7336260005,두산퓨얼셀,KOSPI,,21950,2,-50,-0.23,22000,22200,21600,331761,7266798650,1437587285700,65493726,STK,2018-07-01,5406.0,306.030337
62,352480,KR7352480008,씨앤씨인터내셔널,KOSDAQ,벤처기업부,104200,1,1800,1.76,101400,107400,101200,99686,10482769500,1043452652200,10013941,KSQ,2018-07-01,27465.0,279.391953
9,403870,KR7403870009,HPSP,KOSDAQ GLOBAL,중견기업부,38950,1,900,2.37,37750,39450,37750,858674,33409849300,3230284908800,82934144,KSQ,2018-07-01,10813.0,260.214557
